In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/2021-graph-vs-time-series/train_v2.csv
/kaggle/input/2021-graph-vs-time-series/test_v2.csv
/kaggle/input/2021-graph-vs-time-series/train.csv
/kaggle/input/2021-graph-vs-time-series/test.csv


# Baseline

In [2]:
# 讀資料
train = pd.read_csv('/kaggle/input/2021-graph-vs-time-series/train_v2.csv', index_col=0)
test = pd.read_csv('/kaggle/input/2021-graph-vs-time-series/test_v2.csv', index_col=0)
print(f'train 資料共 {train.shape[0]} 筆')
print(f'test 資料共 {test.shape[0]} 筆')
print(f'train 資料欄位：{train.columns}')

/opt/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


train 資料共 1006717 筆
test 資料共 421665 筆
train 資料欄位：Index(['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'fraud_ind', 'hcefg', 'insfg', 'iterm', 'locdt',
       'loctm', 'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', 'txkey',
       'hour', 'dt_dif1_chid', 'dt_dif2_cano', 'dt_dif1_cano_mchno',
       'pct_of_mchno_fraud_1m', 'avg_mchno_fraud_amt_1m',
       'avg_scity_fraud_amt_1w', 'ratio_mchno_notfraud_avg_amt_distance_1m',
       'ratio_cumcnt_cano_ecfg_w', 'ratio_cumsum_bacno_d'],
      dtype='object')


In [3]:
# 照時間排序，目的：切train/valid
train = train.sort_values(by=['locdt','loctm','bacno']).reset_index(drop=True)

In [4]:
Y = train['fraud_ind']
X = train.iloc[:,(-10):]

In [5]:
display(X.describe())
display(X.isnull().sum())

,hour,dt_dif1_chid,dt_dif2_cano,dt_dif1_cano_mchno,pct_of_mchno_fraud_1m,avg_mchno_fraud_amt_1m,avg_scity_fraud_amt_1w,ratio_mchno_notfraud_avg_amt_distance_1m,ratio_cumcnt_cano_ecfg_w,ratio_cumsum_bacno_d
count,1.006717e+06,1.006717e+06,1.006717e+06,1.006717e+06,1.006717e+06,1.006717e+06,1.006717e+06,1.006717e+06,1.006717e+06,1.006717e+06
mean,1.433051e+01,2.595823e+03,1.144425e+03,4.831213e+02,1.265799e-02,1.190764e+03,5.119724e+03,5.790267e+06,1.057681e+04,1.730814e+07
std,5.205202e+00,8.375091e+03,6.074580e+03,3.881298e+03,6.832390e-02,4.969185e+03,8.368379e+03,2.582179e+07,9.287047e+04,3.919981e+07
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00
25%,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-9.808583e-01,0.000000e+00,1.279215e+00
50%,1.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.457453e+03,-8.005353e-01,0.000000e+00,3.399804e+00
75%,1.800000e+01,2.500000e+01,0.000000e+00,0.000000e+00,6.168270e-05,0.000000e+00,6.268214e+03,9.976221e-01,9.230741e-01,1.657059e+01
max,2.300000e+01,8.631200e+04,8.631100e+04,8.631200e+04,1.000000e+00,3.610070e+04,7.498529e+04,5.833700e+08,7.700000e+06,1.320937e+09


hour                                        0
dt_dif1_chid                                0
dt_dif2_cano                                0
dt_dif1_cano_mchno                          0
pct_of_mchno_fraud_1m                       0
avg_mchno_fraud_amt_1m                      0
avg_scity_fraud_amt_1w                      0
ratio_mchno_notfraud_avg_amt_distance_1m    0
ratio_cumcnt_cano_ecfg_w                    0
ratio_cumsum_bacno_d                        0
dtype: int64

In [6]:
# 切出 train/valid
num = int(np.ceil(len(X)*0.8))
x_train = X.iloc[:num, :]
x_valid = X.iloc[num:, :]
y_train = Y[:num]
y_valid = Y[num:]

In [7]:
from lightgbm import LGBMClassifier
lgbc = LGBMClassifier()
lgbc.fit(x_train, y_train)

LGBMClassifier()

In [8]:
from sklearn.metrics import f1_score

y_valid_pred  = lgbc.predict(x_valid).astype(int)
display(f1_score(y_valid, y_valid_pred))

0.46718506998444786

In [9]:
from sklearn.metrics import accuracy_score

# evaluate predictions
predictions = [round(value) for value in y_valid_pred]
accuracy = accuracy_score(y_valid, predictions)
print(f"Accuracy: {accuracy * 100.0:.2f}%")

Accuracy: 99.15%


In [10]:
y_pred_test = lgbc.predict(test[X.columns]).astype(int)
submit_df = pd.DataFrame({'txkey':test.txkey, 'fraud_ind':y_pred_test})
display(submit_df.shape)
display(submit_df.fraud_ind.value_counts())

(421665, 2)

0    420744
1       921
Name: fraud_ind, dtype: int64

In [11]:
submit_df.to_csv('baseline.csv',index=False)

# Graph

In [12]:
import networkx as nx

In [13]:
# 讀資料
g_train = pd.read_csv('/kaggle/input/2021-graph-vs-time-series/train.csv')
g_test = pd.read_csv('/kaggle/input/2021-graph-vs-time-series/test.csv')
g_data = g_train.append(g_test)
print(f'g_data 資料共 {g_data.shape[0]} 筆')
print(f'g_data 資料欄位：{g_data.columns}')

g_data 資料共 1943452 筆
g_data 資料欄位：Index(['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'fraud_ind', 'hcefg', 'insfg', 'iterm', 'locdt',
       'loctm', 'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', 'txkey'],
      dtype='object')


In [14]:
# 整理成 edgelist 樣貌
node_col = ['bacno','mchno']
g_node = g_data[node_col].drop_duplicates().reset_index(drop=True)
display(g_node.head(2))
g_node.shape

,bacno,mchno
0,113261,59333
1,134508,0


(1021484, 2)

In [15]:
# 讓 bacno 和 mchno 節點編號不同
print(g_node.bacno.max())
g_node['mchno'] = g_node.mchno+200000
min(g_node.mchno)

163886


200000

In [16]:
G = nx.convert_matrix.from_pandas_edgelist(g_node, source='bacno', target='mchno')
# G is an undirected graph
type(G)

networkx.classes.graph.Graph

In [17]:
print(g_node.shape)
print(g_node.bacno.nunique())
print(g_node.mchno.nunique())
print(g_node.bacno.nunique()+g_node.mchno.nunique())

(1021484, 2)
163185
102783
265968


In [18]:
num_edges = G.number_of_edges()
num_nodes = G.number_of_nodes()
print(f'number of nodes={num_nodes}; number of edges={num_edges}')

number of nodes=265968; number of edges=1021484


In [19]:
largest_cc = max(nx.connected_components(G), key=len)
[len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True)][:5]

[263543, 24, 20, 15, 10]

In [20]:
(g_node.head())
nx.write_edgelist(G, "test.edgelist")

# deepwalk

In [21]:
!pip install deepwalk

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [22]:
# 執行deepwalk: 參數都設小一點不然跑不動
!deepwalk --format edgelist --input test.edgelist --workers 2 --number-walks 1 --representation-size 2 --walk-length 5 --window-size 100 --output test_output.embeddings

Number of nodes: 265968
Number of walks: 265968
Data size (walks*length): 1329840
Walking...
Training...


In [23]:
# 執行deepwalk: 參數都設小一點不然跑不動
!deepwalk --format edgelist --input test.edgelist --workers 24 --number-walks 5 --representation-size 2 --walk-length 5 --window-size 100 --output test_output2.embeddings

Number of nodes: 265968
Number of walks: 1329840
Data size (walks*length): 6649200
Walking...
Training...


In [24]:
# 執行deepwalk: 參數都設小一點不然跑不動
!deepwalk --format edgelist --input test.edgelist --workers 24 --number-walks 10 --representation-size 2 --walk-length 2 --window-size 100 --output test_output3.embeddings

Number of nodes: 265968
Number of walks: 2659680
Data size (walks*length): 5319360
Walking...
Training...


In [25]:
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score

def test_submit(embed_file='test_output.embeddings'):
    # embedding 結果讀入
    f = open(embed_file)
    k = f.readlines()
    
    # 整理 embedding vector
    pre_node = [int(x.split(' ')[0]) for x in k[1:]]
    x1 = [(float(x.split(' ')[1])) for x in k[1:]]
    x2 = [float(x.split(' ')[2].split('\n')[0]) for x in k[1:]]
    g_node_embed = pd.DataFrame({'node':pre_node, 'x1':x1, 'x2':x2})
    # bacno 的 embedding
    g_node2 = g_node.merge(g_node_embed, how='left', left_on='bacno', right_on='node')
    g_node2 = g_node2.rename(columns={'x1':'bacno_x1', 'x2':'bacno_x2'})
    # mchno 的 embedding
    g_node2 = g_node2.merge(g_node_embed, how='left', left_on='mchno', right_on='node')
    g_node2 = g_node2.rename(columns={'x1':'mchno_x1', 'x2':'mchno_x2'})
    
    g_node2 = g_node2.drop(columns=['node_x', 'node_y']).drop_duplicates()
    g_node2['mchno'] = g_node2['mchno']-200000
    print(g_node2.shape)
    display(g_node2.head(2))
    print(g_node2.bacno.nunique()+g_node.mchno.nunique())

    train2 = train.merge(g_node2, how='left', on=['bacno','mchno'])
    test2 = test.merge(g_node2, how='left', on=['bacno','mchno'])
    print(f'train2 資料共 {train2.shape[0]} 筆')
    print(f'test2 資料共 {test2.shape[0]} 筆')
    print(f'train2 資料欄位：{train2.columns}')

    train2 = train2.sort_values(by=['locdt','loctm','bacno']).reset_index(drop=True)

    Y2 = train2['fraud_ind']
    X2 = train2.iloc[:,(-14):]

    # 切出 train/valid
    num = int(np.ceil(len(X2)*0.8))
    x_train2 = X2.iloc[:num, :]
    x_valid2 = X2.iloc[num:, :]
    y_train2 = Y2[:num]
    y_valid2 = Y2[num:]

    # train lgbm
    lgbc2 = LGBMClassifier()
    lgbc2.fit(x_train2, y_train2)

    # 評估
    y_valid_pred2  = lgbc2.predict(x_valid2).astype(int)
    display(f1_score(y_valid2, y_valid_pred2))

    # prediction
    y_pred_test2 = lgbc2.predict(test2[X2.columns]).astype(int)
    submit_df2 = pd.DataFrame({'txkey':test2.txkey, 'fraud_ind':y_pred_test2})
    display(submit_df2.shape)
    display(submit_df2.fraud_ind.value_counts())

    return submit_df2

In [26]:
test1 = test_submit(embed_file='test_output.embeddings')
test1.to_csv('deepwalk_1.csv',index=False)

(1021484, 6)


,bacno,mchno,bacno_x1,bacno_x2,mchno_x1,mchno_x2
0,113261,59333,1.391831,1.092911,1.082311,1.558644
1,134508,0,0.464692,0.344654,0.945098,1.494870


265968
train2 資料共 1006717 筆
test2 資料共 421665 筆
train2 資料欄位：Index(['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'fraud_ind', 'hcefg', 'insfg', 'iterm', 'locdt',
       'loctm', 'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', 'txkey',
       'hour', 'dt_dif1_chid', 'dt_dif2_cano', 'dt_dif1_cano_mchno',
       'pct_of_mchno_fraud_1m', 'avg_mchno_fraud_amt_1m',
       'avg_scity_fraud_amt_1w', 'ratio_mchno_notfraud_avg_amt_distance_1m',
       'ratio_cumcnt_cano_ecfg_w', 'ratio_cumsum_bacno_d', 'bacno_x1',
       'bacno_x2', 'mchno_x1', 'mchno_x2'],
      dtype='object')


0.45305164319248825

(421665, 2)

0    419583
1      2082
Name: fraud_ind, dtype: int64

In [27]:
test2 = test_submit(embed_file='test_output2.embeddings')
test2.to_csv('deepwalk_2.csv',index=False)

(1021484, 6)


,bacno,mchno,bacno_x1,bacno_x2,mchno_x1,mchno_x2
0,113261,59333,0.993268,0.773528,0.823405,0.845161
1,134508,0,0.778620,1.056310,0.773773,0.764064


265968
train2 資料共 1006717 筆
test2 資料共 421665 筆
train2 資料欄位：Index(['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'fraud_ind', 'hcefg', 'insfg', 'iterm', 'locdt',
       'loctm', 'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', 'txkey',
       'hour', 'dt_dif1_chid', 'dt_dif2_cano', 'dt_dif1_cano_mchno',
       'pct_of_mchno_fraud_1m', 'avg_mchno_fraud_amt_1m',
       'avg_scity_fraud_amt_1w', 'ratio_mchno_notfraud_avg_amt_distance_1m',
       'ratio_cumcnt_cano_ecfg_w', 'ratio_cumsum_bacno_d', 'bacno_x1',
       'bacno_x2', 'mchno_x1', 'mchno_x2'],
      dtype='object')


0.4563882063882064

(421665, 2)

0    419936
1      1729
Name: fraud_ind, dtype: int64

In [28]:
test2 = test_submit(embed_file='test_output3.embeddings')
test2.to_csv('deepwalk_3.csv',index=False)

(1021484, 6)


,bacno,mchno,bacno_x1,bacno_x2,mchno_x1,mchno_x2
0,113261,59333,-0.324066,-0.029748,-1.986924,-1.783653
1,134508,0,-0.122314,0.118549,-1.813804,-1.672819


265968
train2 資料共 1006717 筆
test2 資料共 421665 筆
train2 資料欄位：Index(['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'fraud_ind', 'hcefg', 'insfg', 'iterm', 'locdt',
       'loctm', 'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', 'txkey',
       'hour', 'dt_dif1_chid', 'dt_dif2_cano', 'dt_dif1_cano_mchno',
       'pct_of_mchno_fraud_1m', 'avg_mchno_fraud_amt_1m',
       'avg_scity_fraud_amt_1w', 'ratio_mchno_notfraud_avg_amt_distance_1m',
       'ratio_cumcnt_cano_ecfg_w', 'ratio_cumsum_bacno_d', 'bacno_x1',
       'bacno_x2', 'mchno_x1', 'mchno_x2'],
      dtype='object')


0.45172824791418353

(421665, 2)

0    420788
1       877
Name: fraud_ind, dtype: int64

## node2vec

In [29]:
# !pip install node2vec

In [30]:
# import networkx as nx
# from node2vec import Node2Vec

# # Create a graph
# graph = G

# # Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
# node2vec = Node2Vec(graph, dimensions=2, walk_length=2, num_walks=5, workers=1, temp_folder='/')  # Use temp_folder for big graphs

# # Embed nodes
# model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed, `dimensions` and `workers` are automatically passed (from the Node2Vec constructor)

# # Look for most similar nodes
# model.wv.most_similar('2')  # Output node names are always strings

# # Save embeddings for later use
# model.wv.save_word2vec_format('EMBEDDING_FILENAME')

# # Save model for later use
# model.save('EMBEDDING_MODEL_FILENAME')

# # Embed edges using Hadamard method
# from node2vec.edges import HadamardEmbedder

# edges_embs = HadamardEmbedder(keyed_vectors=model.wv)

### CS224W
- https://colab.research.google.com/drive/1b9rZIjD7MUEKwYbXZc3dchTBTpzdrvpd?usp=sharing
- https://zhangruochi.com/CS224W-Colab-1/2021/01/22/

In [31]:
# import torch
# print(torch.__version__)

### Question 5: Getting the edge list of the karate club network and transform it into torch.LongTensor. What is the torch.sum value of pos_edge_index tensor? 

In [32]:
# def graph_to_edge_list(G):
#   # TODO: Implement the function that returns the edge list of
#   # an nx.Graph. The returned edge_list should be a list of tuples
#   # where each tuple is a tuple representing an edge connected 
#   # by two nodes.

#     edge_list = []

#   ############# Your code here ############

#     edge_list = list(G.edges())
#   #########################################

#     return edge_list

# def edge_list_to_tensor(edge_list):
#     # TODO: Implement the function that transforms the edge_list to
#     # tensor. The input edge_list is a list of tuples and the resulting
#     # tensor should have the shape [2 x len(edge_list)].

#     edge_index = torch.tensor([])

#     ############# Your code here ############
#     edge_index = torch.tensor(edge_list, dtype = torch.long).permute((1,0))

#     #########################################

#     return edge_index

# pos_edge_list = graph_to_edge_list(G)
# pos_edge_index = edge_list_to_tensor(pos_edge_list)
# print("The pos_edge_index tensor has shape {}".format(pos_edge_index.shape))
# print("The pos_edge_index tensor has sum value {}".format(torch.sum(pos_edge_index)))

### Question 6: Please implement following function that samples negative edges. Then you will answer which edges (edge_1 to edge_5) can be negative ones in the karate club network?

In [33]:
# import random

# def sample_negative_edges(G, num_neg_samples):
#     # TODO: Implement the function that returns a list of negative edges.
#     # The number of sampled negative edges is num_neg_samples. You do not
#     # need to consider the corner case when the number of possible negative edges
#     # is less than num_neg_samples. It should be ok as long as your implementation 
#     # works on the karate club network. In this implementation, self loop should 
#     # not be considered as either a positive or negative edge. Also, notice that 
#     # the karate club network is an undirected graph, if (0, 1) is a positive 
#     # edge, do you think (1, 0) can be a negative one?

#     neg_edge_list = []

#     pos_set = set(G.edges())
#     visited_set = set()

#     ############# Your code here ############
#     for n_i in G.nodes():
#         for n_j in G.nodes():
#             if n_i == n_j or (n_i,n_j) in pos_set or (n_j,n_i) in pos_set or (n_i,n_j) in visited_set or (n_j, n_i) is visited_set:
#                 continue
#             neg_edge_list.append((n_i,n_j))
#             visited_set.add((n_i,n_j))
#             visited_set.add((n_j,n_i))
#             if len(neg_edge_list) == num_neg_samples:
#                 break

#     #########################################

#     return neg_edge_list

# # Sample 78 negative edges
# neg_edge_list = sample_negative_edges(G, len(pos_edge_list))

# # Transform the negative edge list to tensor
# neg_edge_index = edge_list_to_tensor(neg_edge_list)
# print("The neg_edge_index tensor has shape {}".format(neg_edge_index.shape))

# # Which of following edges can be negative ones?
# edge_1 = (7, 1)
# edge_2 = (1, 33)
# edge_3 = (33, 22)
# edge_4 = (0, 4)
# edge_5 = (4, 2)

# ############# Your code here ############
# ## Note:
# ## 1: For each of the 5 edges, print whether it can be negative edge

# #########################################

### 3 Node Emebedding Learning

In [34]:
# import torch
# import torch.nn as nn
# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA

# print(torch.__version__)

In [35]:
# # Initialize an embedding layer
# # Suppose we want to have embedding for 4 items (e.g., nodes)
# # Each item is represented with 8 dimensional vector

# emb_sample = nn.Embedding(num_embeddings=4, embedding_dim=8)
# print('Sample embedding layer: {}'.format(emb_sample))

In [36]:
# # Select an embedding in emb_sample
# id = torch.LongTensor([1])
# print(emb_sample(id))

# # Select multiple embeddings
# ids = torch.LongTensor([1, 3])
# print(emb_sample(ids))

# # Get the shape of the embedding weight matrix
# shape = emb_sample.weight.data.shape
# print(shape)

# # Overwrite the weight to tensor with all ones
# emb_sample.weight.data = torch.ones(shape)

# # Let's check if the emb is indeed initilized
# ids = torch.LongTensor([0, 3])
# print(emb_sample(ids))

In [37]:
# # Please do not change / reset the random seed
# torch.manual_seed(1)

# def create_node_emb(num_node=34, embedding_dim=16):
#   # TODO: Implement this function that will create the node embedding matrix.
#   # A torch.nn.Embedding layer will be returned. You do not need to change 
#   # the values of num_node and embedding_dim. The weight matrix of returned 
#   # layer should be initialized under uniform distribution. 

#   emb = None

#   ############# Your code here ############
#   emb = torch.nn.Embedding(num_embeddings = num_node, embedding_dim = embedding_dim)

#   #########################################

#   return emb

# emb = create_node_emb()
# print(emb.weight.data.shape)
# ids = torch.LongTensor([0, 3])

# # Print the embedding layer
# print("Embedding: {}".format(emb))

# # An example that gets the embeddings for node 0 and 3
# print(emb(ids))

In [38]:
# from torch.optim import SGD

# def accuracy(pred, label):
#   # TODO: Implement the accuracy function. This function takes the 
#   # pred tensor (the resulting tensor after sigmoid) and the label 
#   # tensor (torch.LongTensor). Predicted value greater than 0.5 will 
#   # be classified as label 1. Else it will be classified as label 0.
#   # The returned accuracy should be rounded to 4 decimal places. 
#   # For example, accuracy 0.82956 will be rounded to 0.8296.

#   accu = 0.0

#   ############# Your code here ############
#   pred_label = pred.ge(0.5) 
#   accu = torch.sum(pred_label == label) / label.shape[0]
#   accu = accu.item()

#   #########################################

#   return round(accu,4)

# def train(emb, loss_fn, sigmoid, train_label, train_edge):
#   # TODO: Train the embedding layer here. You can also change epochs and 
#   # learning rate. In general, you need to implement: 
#   # (1) Get the embeddings of the nodes in train_edge
#   # (2) Dot product the embeddings between each node pair
#   # (3) Feed the dot product result into sigmoid
#   # (4) Feed the sigmoid output into the loss_fn
#   # (5) Print both loss and accuracy of each epoch 
#   # (as a sanity check, the loss should decrease during training)

#     ############# Your code here ############
#     epochs = 1000
#     learning_rate = 0.5

#     optimizer = SGD(emb.parameters(), lr=learning_rate, momentum=0.9)

#     for i in range(epochs):
#       optimizer.zero_grad()  # Clear gradients.

#       product = torch.sum(torch.mul(emb(train_edge[0]),emb(train_edge[1])), axis = 1)
#       pred = torch.sigmoid(product)
#       loss = loss_fn(pred, train_label)
#       loss.backward()  # Derive gradients.
#       optimizer.step()  # Update parameters based on gradients.

    
#       with torch.no_grad():
#         accu = accuracy(pred, train_label)
#         if i % 100 == 0:
#           visualize_emb(emb)
#         print("loss: {}, accuracy: {}".format(loss.item(), accu))
  

#     #########################################

# loss_fn = nn.BCELoss()
# sigmoid = nn.Sigmoid()

# # Generate the positive and negative labels
# pos_label = torch.ones(pos_edge_index.shape[1], )
# neg_label = torch.zeros(neg_edge_index.shape[1], )

# # Concat positive and negative labels into one tensor
# train_label = torch.cat([pos_label, neg_label], dim=0)

# # Concat positive and negative edges into one tensor
# # Since the network is very small, we do not split the edges into val/test sets
# train_edge = torch.cat([pos_edge_index, neg_edge_index], dim=1)

# train(emb, loss_fn, sigmoid, train_label, train_edge)

### DeepWalk
- https://github.com/palash1992/GEM

### References [GraphSage]
- GitHub
https://github.com/FighterLYL/GraphNeuralNetwork/tree/master/chapter7